In [ ]:
# install huggingface and datasets
!pip install -q datasets transformers
!pip install torch
!pip install pandas
!pip install pyarrow

     |████████████████████████████████| 346 kB 32.1 MB/s 
     |████████████████████████████████| 4.2 MB 58.8 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 1.1 MB 54.1 MB/s 
     |████████████████████████████████| 84 kB 3.5 MB/s 
     |████████████████████████████████| 140 kB 80.3 MB/s 
     |████████████████████████████████| 212 kB 70.4 MB/s 
     |████████████████████████████████| 127 kB 76.3 MB/s 
     |████████████████████████████████| 596 kB 72.0 MB/s 
     |████████████████████████████████| 6.6 MB 55.0 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 75.7 MB/s 
     |████████████████████████████████| 144 kB 73.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is

In [ ]:
from datasets import load_dataset, ClassLabel
from transformers import BertForTokenClassification, BertTokenizer, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn import metrics


import pandas as pd
import datasets
from datasets.features import ClassLabel

In [ ]:
torch.cuda.is_available()
device = torch.device("cuda")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/e_ML4NLP2/v2.1/'

Mounted at /content/drive
ajmc  hipe2020	letemps  newseye  sonar  topres19th


In [ ]:
train_path_1 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/sv/HIPE-2022-v2.1-newseye-train-sv.tsv'
dev_path_1 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/sv/HIPE-2022-v2.1-newseye-dev-sv.tsv'
test_path_1 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/sv/HIPE-2022-v2.1-newseye-test_allmasked-sv.tsv'

train_path_2 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/fi/HIPE-2022-v2.1-newseye-train-fi.tsv'
dev_path_2 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/fi/HIPE-2022-v2.1-newseye-dev-fi.tsv'
test_path_2 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/fi/HIPE-2022-v2.1-newseye-test_allmasked-fi.tsv'

train_path_3 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/de/HIPE-2022-v2.1-newseye-train-de.tsv'
dev_path_3 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/de/HIPE-2022-v2.1-newseye-dev-de.tsv'
test_path_3 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/de/HIPE-2022-v2.1-newseye-test_allmasked-de.tsv'

train_path_4 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/fr/HIPE-2022-v2.1-newseye-train-fr.tsv'
dev_path_4 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/fr/HIPE-2022-v2.1-newseye-dev-fr.tsv'
test_path_4 = '/content/drive/MyDrive/e_ML4NLP2/v2.1/newseye/fr/HIPE-2022-v2.1-newseye-test_allmasked-fr.tsv'


model = 'dbmdz/bert-base-historic-multilingual-cased'
#model = 'setu4993/LaBSE'
#model = 'bert-base-multilingual-cased'
#model = 'bert-base-multilingual-uncased'
#model = 'distilbert-base-multilingual-cased'

In [ ]:
# import dataset from cloned git repo
def load_dataset(path):
    df = pd.read_csv(path, sep='\t', skip_blank_lines=False, engine='python', quoting=3)
    # error_bad_lines=False, 
    return df


tsv_train_1 = load_dataset(train_path_1)
tsv_dev_1 = load_dataset(dev_path_1)
tsv_test_1 = load_dataset(test_path_1)

tsv_train_2 = load_dataset(train_path_2)
tsv_dev_2 = load_dataset(dev_path_2)
tsv_test_2 = load_dataset(test_path_2)

tsv_train_3 = load_dataset(train_path_3)
tsv_dev_3 = load_dataset(dev_path_3)
tsv_test_3 = load_dataset(test_path_3)

tsv_train_4 = load_dataset(train_path_4)
tsv_dev_4 = load_dataset(dev_path_4)
tsv_test_4 = load_dataset(test_path_4)

tsv_train = pd.concat([tsv_train_1, tsv_train_2, tsv_train_3, tsv_train_4])
tsv_dev = pd.concat([tsv_dev_1, tsv_dev_2, tsv_dev_3, tsv_dev_4])

del tsv_train_1
del tsv_train_2
del tsv_train_3
del tsv_train_4
del tsv_dev_1
del tsv_dev_2
del tsv_dev_3
del tsv_dev_4

In [ ]:
def simple_preprocess(dataframe):
    # Add end_of_document token in df
    dataframe = dataframe.dropna(subset=['TOKEN'])

    # Filter out metadata rows beginning with #
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('#')]
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('\t')]

    #transforming nan var from Float to string to use in (***)
    dataframe.MISC = dataframe.MISC.fillna('')

    return dataframe

In [ ]:
tsv_train = simple_preprocess(tsv_train)
tsv_dev = simple_preprocess(tsv_dev)

tsv_test_1 = simple_preprocess(tsv_test_1)
tsv_test_2 = simple_preprocess(tsv_test_2)
tsv_test_3 = simple_preprocess(tsv_test_3)
tsv_test_4 = simple_preprocess(tsv_test_4)
tsv_dev.head(100)

,TOKEN,NE-COARSE-LIT,NE-COARSE-METO,NE-FINE-LIT,NE-FINE-METO,NE-FINE-COMP,NE-NESTED,NEL-LIT,NEL-METO,MISC
12,Fondekorationen,O,_,O,_,_,O,_,_,NoSpaceAfter
13,",",O,_,O,_,_,O,_,_,_
14,föreställande,O,_,O,_,_,O,_,_,NoSpaceAfter
15,Kuddnäs,B-LOC,_,O,_,_,O,NIL,_,NoSpaceAfter
16,",",I-LOC,_,O,_,_,O,NIL,_,_
...,...,...,...,...,...,...,...,...,...,...
107,kort,O,_,O,_,_,O,_,_,NoSpaceAfter
108,före,O,_,O,_,_,O,_,_,_
109,senaste,O,_,O,_,_,O,_,_,_
110,julhälg,O,_,O,_,_,O,_,_,_


In [ ]:
label_set = tsv_train['NE-COARSE-LIT'].unique()
# define the label mapping for NER
label_list = label_set.tolist()
label_list.append('_')
label_num = len(label_list)

labels = ClassLabel(num_classes=label_num, names=label_list)

labels

ClassLabel(num_classes=10, names=['O', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-HumanProd', 'I-HumanProd', '_'], id=None)

In [ ]:
def create_huggingface_file(dataframe):
    import pyarrow as pa
    import re
    from datasets import Dataset

    #creating dataset in json
    hug_out = []
    idx = 0 
    items = {'id': idx,'words':[ ], 'ner': [ ]}
    hug_out.append(items)
    for index, row in dataframe.iterrows():
        if  not re.search(r'EndOfSentence', row['MISC']):
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
        else:
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
            idx += 1
            items = {'id': idx,'words':[ ], 'ner': [ ]}
            hug_out.append(items)
    #filter hug_out out, delete items which has len(words) > 380
    #hug_out = filter(lambda x: len(x['words']) < 380, hug_out)
    #json to df
    hug_out = pd.DataFrame(hug_out)

    # delete all sentences that are too long
    #hug_out = hug_out[hug_out['words'].map(len) < 512] #why does not work? QA

    ### convert to Huggingface dataset
    hug_out = Dataset(pa.Table.from_pandas(hug_out))

    return hug_out

In [ ]:
train = create_huggingface_file(tsv_train).shuffle(seed=42)
val = create_huggingface_file(tsv_dev)

test_1 = create_huggingface_file(tsv_test_1)
test_2 = create_huggingface_file(tsv_test_2)
test_3 = create_huggingface_file(tsv_test_3)
test_4 = create_huggingface_file(tsv_test_4)


#look at training data
for i in range(10):
  print(train[i])

print(len(train))
print(len(val))

{'id': 16441, 'words': ['Hr', '.', 'Pantl', '.'], 'ner': [0, 0, 0, 0]}
{'id': 23980, 'words': ['Marie', '.'], 'ner': [3, 0]}
{'id': 7362, 'words': [']', 'Eine', 'große', 'Anzahl', 'von', 'Leidtragenden', 'erwies', 'heute', 'dem', 'Chef=Arzte', 'des', 'Officierstöchter=', '[Stabsarzt', 'Dr', '.', 'Rammel', '.'], 'ner': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0]}
{'id': 7923, 'words': ['—', '356', '.'], 'ner': [0, 0, 0]}
{'id': 6416, 'words': ['Gal', '.', 'Karl', '-', 'Ludwigb', '.', '200', 'fl', '.', 'CM', '.'], 'ner': [5, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0]}
{'id': 2099, 'words': ['(', 'Rjetsh', ')', 'Tänne', 'ilmoitetaan', 'Konstan=', 'tinopelista', ',', 'että', 'sen', 'johdosta', 'että', 'Tur=', 'kin', 'armeijalta', 'puuttuu', 'aseita', 'ja', 'sota=', 'tarpeita', ',', 'owat', 'Konstantinopolin', 'wiran=', 'omaiset', 'takawarikoineet', 'kaikki', 'sekä', 'ulko=', 'että', 'kotimaiset', 'asewarastoi', 'Konstanti=', 'nopolissa', '.', 'Samaan', 'aikaan', 'on', 'poliisi', 'toimeenpannut'

In [ ]:
type(train)

datasets.arrow_dataset.Dataset

In [ ]:
'''LOADERSCRIPT = train_path = '/content/drive/MyDrive/e_ML4NLP2/dataloader.py'
MODEL = "dbmdz/bert-base-german-europeana-cased"'''

'LOADERSCRIPT = train_path = \'/content/drive/MyDrive/e_ML4NLP2/dataloader.py\'\nMODEL = "dbmdz/bert-base-german-europeana-cased"'

In [ ]:
'''de_total = load_dataset(LOADERSCRIPT, "de")'''

'de_total = load_dataset(LOADERSCRIPT, "de")'

In [ ]:
'''train = de_total["train"]
val = de_total["validation"]
test = de_total["test"]'''

'train = de_total["train"]\nval = de_total["validation"]\ntest = de_total["test"]'

In [ ]:
'''tiny_shuffle_train =  train.shuffle(seed=42).select(range(100))
tiny_train = tiny_shuffle_train.select(range(0,80))
tiny_shuffle_test = test.shuffle(seed=42).select(range(100))
tiny_test = tiny_shuffle_test.select(range(80,100))'''

'tiny_shuffle_train =  train.shuffle(seed=42).select(range(100))\ntiny_train = tiny_shuffle_train.select(range(0,80))\ntiny_shuffle_test = test.shuffle(seed=42).select(range(100))\ntiny_test = tiny_shuffle_test.select(range(80,100))'

In [ ]:
'''tiny_shuffle_train[10]'''

'tiny_shuffle_train[10]'

In [ ]:
'''labels = train.info.features["NE_COARSE_LIT"].feature
labels'''

'labels = train.info.features["NE_COARSE_LIT"].feature\nlabels'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
def tokenize_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]
    tokenized_ner = []

    for sent_words, sent_nes in zip(tokenized_words, batch["ner"]):
        tokenized_ner.append(
            [labels.str2int("O")]  # BOS symbol
            + [
                ne
                for subwords, ne in zip(sent_words, sent_nes)
                for _ in range(len(subwords))
            ]
            + [labels.str2int("O")]  # EOS symbol
        )

    # Padding with "O"
    tokenized["labels"] = [(ner + [labels.str2int("O")] * (100 - len(ner)))[:100] for ner in tokenized_ner]
    tokenized["subwords"] = tokenized_words
    return tokenized

def tokenize_nolabel_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]

    tokenized["subwords"] = tokenized_words
    return tokenized

#tiny_train_tokenized = tiny_train.map(tokenize_func, batched=True, batch_size=50)
#tiny_test_tokenized = tiny_test.map(tokenize_nolabel_func, batched=True, batch_size=50)

train_tokenized = train.map(tokenize_func, batched=True, batch_size=50)
val_tokenized = val.map(tokenize_func, batched=True, batch_size=50)

test_tokenized_1 = test_1.map(tokenize_nolabel_func, batched=True, batch_size=50)
test_tokenized_2 = test_2.map(tokenize_nolabel_func, batched=True, batch_size=50)
test_tokenized_3 = test_3.map(tokenize_nolabel_func, batched=True, batch_size=50)
test_tokenized_4 = test_4.map(tokenize_nolabel_func, batched=True, batch_size=50)

  0%|          | 0/603 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/67 [00:00<?, ?ba/s]

  0%|          | 0/51 [00:00<?, ?ba/s]

In [ ]:
'''# train for 1 epochs, evaluate after each epoch
training_args = TrainingArguments(
    "test_trainer",
    num_train_epochs=3,
    evaluation_strategy="epoch"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
)
trainer.train()'''

'# train for 1 epochs, evaluate after each epoch\ntraining_args = TrainingArguments(\n    "test_trainer",\n    num_train_epochs=3,\n    evaluation_strategy="epoch"\n)\n\n\ntrainer = Trainer(\n    model=model,\n    args=training_args,\n    train_dataset=tokenized_train,\n    eval_dataset=tokenized_dev,\n)\ntrainer.train()'

In [ ]:
training_args = TrainingArguments(
    output_dir = "content/drive/MyDrive/e_ML4NLP2/"
)

In [ ]:
model = BertForTokenClassification.from_pretrained(model, num_labels=labels.num_classes)
trainer = Trainer(model=model, args=training_args, train_dataset=train_tokenized)
trainer.train()

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['distilbert.embeddings.position_embeddings.weight', 'distilbert.transformer.layer.4.output_layer_norm.bias', 'distilbert.transformer.layer.2.attention.out_lin.weight', 'vocab_transform.bias', 'distilbert.transformer.layer.3.attention.q_lin.bias', 'distilbert.transformer.layer.4.output_layer_norm.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.transformer.layer.4.attention.q_lin.bias', 'distilbert.transformer.layer.2.attention.k_lin.bias', 'distilbert.transformer.layer.1.sa_layer_norm.bias', 'distilbert.transformer.layer.2.ffn.lin2.weight', 'distilbert.transformer.layer.3.attention.v_lin.bias', 'distilbert.transformer.layer.4.attention.out_lin.bias', 'vocab_projector.weight', 'distilbert.transformer.layer.1.attention.v_lin.weight', 'distilbert.transformer.layer.1.ffn.lin2.weight', 'distilbert.transformer.layer.5.attention.v_lin.weight', '

Step,Training Loss
500,0.162500
1000,0.145900
1500,0.122700
2000,0.122900
2500,0.118500
3000,0.109000
3500,0.111200


Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-500
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-500/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-1000
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1000/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-1500
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1500/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-2000
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-2000/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-2000/pytorch_model.bin
Saving model ch

KeyboardInterrupt: ignored

In [ ]:
predictions_1 = trainer.predict(test_tokenized_1)
predictions_2 = trainer.predict(test_tokenized_2)
predictions_3 = trainer.predict(test_tokenized_3)
predictions_4 = trainer.predict(test_tokenized_4)

In [ ]:
def write_outfile(predictions, outfile_name, test_tokenized):
    outfile = open(outfile_name, "w")
    outfile.write("TOKEN\tNE-COARSE-LIT\tNE-COARSE-METO\tNE-FINE-LIT\tNE-FINE-METO\tNE-FINE-COMP\tNE-NESTED\tNEL-LIT\tNEL-METO\tMISC")

    for pred, tinput in zip(predictions.predictions, test_tokenized):
        pred_word_labels = []

        try:
            sent_len = tinput["input_ids"].index(0)
        except ValueError:
            sent_len = len(tinput["input_ids"])
        # Start at 1 to skip pred for cls token
        i = 1

        for word in tinput['subwords']:
        
            word_score = np.zeros_like(pred[0])
            eos_reached = False
            try:
                for subword in word:
                    word_score += pred[i]
                    i += 1
                label = labels.int2str(int(np.argmax(word_score)))
                pred_word_labels.append(label)
            except IndexError:
                pred_word_labels.append("O")

        for token, label in zip(tinput['words'], pred_word_labels):
            outfile.write("\n" + f"{token}\t{label}\t" + ('\t'.join(['_'] * 8)))

    outfile.close()

In [ ]:
write_outfile(predictions_1, 'result_1.tsv', test_tokenized_1)
write_outfile(predictions_2, 'result_2.tsv', test_tokenized_2)
write_outfile(predictions_3, 'result_3.tsv', test_tokenized_3)
write_outfile(predictions_4, 'result_4.tsv', test_tokenized_4)

In [ ]:
accuracy = datasets.load_metric("accuracy")
f1_metric = datasets.load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    refs = labels.flatten()
    predictions = np.argmax(logits, axis=-1)
    print(logits.shape)
    print(predictions.shape)
    pred = predictions.flatten()
    return {
        "accuracy": accuracy.compute(predictions=pred, references=refs)["accuracy"],
        "f1_micro": f1_metric.compute(predictions=pred, references=refs, average="micro")["f1"],
        "f1_macro": f1_metric.compute(predictions=pred, references=refs, average="macro")["f1"],
    }

In [ ]:
Trainer(
    model=model,
    args=training_args,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
).evaluate()